In [1]:
%matplotlib inline

In [2]:
import os
import torch

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [3]:
%load_ext autoreload
%autoreload 2

# load data

In [4]:
from loaders import prepareReportData, tensorsFromPair, normalizeString

In [5]:
input_lang, output_lang, ds = prepareReportData()


reading data..
read 3580 reports
counting words..
counted words:
	 tags 590
	 report 1953


In [6]:
report_pair = ds.__getitem__(100)[:2]
tensorsFromPair((report_pair[0], normalizeString(report_pair[1])), input_lang, output_lang, device, 15)

(tensor([[2],
         [1]], device='cuda:1'),
 tensor([[  2],
         [  3],
         [  9],
         [ 10],
         [ 11],
         [169],
         [  3],
         [  3],
         [ 41],
         [ 94],
         [ 95],
         [ 25],
         [ 20],
         [ 21],
         [ 22]], device='cuda:1'))

# train

In [10]:
from models import EncoderRNN, AttnDecoderRNN

In [11]:
from train_utils import trainIters

In [12]:
%%time
hidden_size = 256
max_length = 30
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, max_length, dropout_p=0.1).to(device)

plot_losses = trainIters(ds, encoder1, attn_decoder1, 100, max_length, input_lang, output_lang, print_every=10)

0m 1s (- 0m 10s) (10 10%) 7.2631
0m 1s (- 0m 7s) (20 20%) 6.5236
0m 2s (- 0m 6s) (30 30%) 6.1840
0m 4s (- 0m 6s) (40 40%) 5.4171
0m 5s (- 0m 5s) (50 50%) 5.5878
0m 6s (- 0m 4s) (60 60%) 4.9356
0m 7s (- 0m 3s) (70 70%) 4.8964
0m 8s (- 0m 2s) (80 80%) 5.2887
0m 8s (- 0m 0s) (90 90%) 4.9547
0m 9s (- 0m 0s) (100 100%) 4.5145
CPU times: user 9.66 s, sys: 236 ms, total: 9.9 s
Wall time: 10.1 s


# eval

In [13]:
from perf import get_av_bleu
from test_utils import evaluateAll, evaluateRandomly

In [14]:
%%time
pred_output = evaluateAll(ds, encoder1, attn_decoder1, max_length, input_lang, output_lang)

CPU times: user 1min 30s, sys: 980 ms, total: 1min 31s
Wall time: 1min 31s


In [17]:
pred_output[0]

('degenerative_change diaphragm',
 ' . mildly elevated right hemidiaphragm . otherwise no acute cardiopulmonary abnormality seen . heart size and mediastinal contours are within normal limits . pulmonary vasculature is unremarkable . no focal airspace consolidation . there is mild elevation right hemidiaphragm . no visible pleural effusion or pneumothorax . there are mild degenerative changes along the thoracic spine .',
 ' . no . . . . is no . . . is no . . . is no . . . is no . . . is no .')

In [15]:
print('BLEU: {:.2%}'.format(
    get_av_bleu(pred_output)))

/home/alex/anaconda3/envs/bcic1/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/alex/anaconda3/envs/bcic1/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/alex/anaconda3/envs/bcic1/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using low

BLEU: 0.00%


# WIP plot learning curve

In [47]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)